In [134]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
import pickle
import random
from sklearn.metrics import accuracy_score

In [135]:
path=r'/kaggle/input/traffic-sign/traffic_Data/DATA'
label_file=r'/kaggle/input/traffic-sign/labels.csv'
batch_size_val=50
steps_per_epoch_val=2000
epochs_val=10
imageDimensions=(32,32,3)
testRatio=0.2
validationRatio=0.2 

In [138]:
count = 0
images = []
classNo = []
myList = os.listdir(path)
print("Total Classes Detected:",len(myList))
noOfClasses=len(myList)
print("Importing Classes.....")
for x in range (0,len(myList)):
    myPicList = os.listdir(path+"/"+str(count))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(count)+"/"+y)
        images.append(curImg)
        classNo.append(count)
    print(count, end =" ")
    count +=1
print(" ")
images = np.array(images, dtype="object")
classNo = np.array(classNo)

Total Classes Detected: 58
Importing Classes.....
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57  


In [139]:
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validationRatio)
 

In [140]:
print("Data Shapes")
print("Train",end = "");print(X_train.shape,y_train.shape)
print("Validation",end = "");print(X_validation.shape,y_validation.shape)
print("Test",end = "");print(X_test.shape,y_test.shape)

Data Shapes
Train(2668,) (2668,)
Validation(668,) (668,)
Test(834,) (834,)


In [141]:
data=pd.read_csv(label_file)
print("data shape ",data.shape,type(data))
 
num_of_samples = []
cols = 5
num_classes = noOfClasses

data shape  (58, 2) <class 'pandas.core.frame.DataFrame'>


In [142]:
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img =cv2.equalizeHist(img)
    return img
def preprocessing(img):
    img = cv2.resize(img , (32,32))
    img = grayscale(img)     
    img = equalize(img) 

    img = img/255            
    return img

In [23]:
a =[]
b = list(map(preprocessing,X_train))
for i in range(len(b)):
    a.append(len(b[i][0]))
a = sorted(a)
a[len(a)-1]  

491

In [143]:
X_train=np.array(list(map(preprocessing,X_train)))  
X_validation=np.array(list(map(preprocessing,X_validation)))
X_test=np.array(list(map(preprocessing,X_test)))
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)
X_validation=X_validation.reshape(X_validation.shape[0],X_validation.shape[1],X_validation.shape[2],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)

In [144]:
dataGen= ImageDataGenerator(width_shift_range=0.1,   
                            height_shift_range=0.1,
                            zoom_range=0.2,  
                            shear_range=0.1,  
                            rotation_range=10)  
dataGen.fit(X_train)
batches= dataGen.flow(X_train,y_train,batch_size=20)
X_batch,y_batch = next(batches)
 

y_train = to_categorical(y_train,noOfClasses)
y_validation = to_categorical(y_validation,noOfClasses)
y_test = to_categorical(y_test,noOfClasses)

In [145]:
def myModel():
    model= Sequential()
    model.add((Conv2D(60,(5,5),input_shape=(imageDimensions[0],imageDimensions[1],1),activation='relu')))  # ADDING MORE CONVOLUTION LAYERS = LESS FEATURES BUT CAN CAUSE ACCURACY TO INCREASE
    model.add((Conv2D(60, (5,5), activation='relu')))
    model.add(MaxPooling2D(pool_size=(2,2)))
 
    model.add((Conv2D(30, (3,3),activation='relu')))
    model.add((Conv2D(30, (3,3), activation='relu')))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.5))
 
    model.add(Flatten())
    model.add(Dense(500,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(noOfClasses,activation='softmax')) 
    model.compile(Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model
 
model = myModel()
print(model.summary())
history=model.fit(dataGen.flow(X_train,y_train,batch_size=32),epochs=10,validation_data=(X_validation,y_validation),shuffle=1)
#history = model.fit(X_train , batch_size = 32  ,  epochs = epochs_val )

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_64 (Conv2D)              │ (None, 28, 28, 60)     │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_65 (Conv2D)              │ (None, 24, 24, 60)     │        90,060 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 12, 12, 60)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_66 (Conv2D)              │ (None, 10, 10, 30)     │        16,230 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_67 (Conv2D)              │ (None, 8, 8, 30)       │         8,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 4, 4, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_16 (Flatten)            │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 500)            │       240,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_33 (Dropout)            │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 58)             │        29,058 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 385,538 (1.47 MB)

 Trainable params: 385,538 (1.47 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


84/84 ━━━━━━━━━━━━━━━━━━━━ 15s 149ms/step - accuracy: 0.0766 - loss: 3.8093 - val_accuracy: 0.1392 - val_loss: 3.4516
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 152ms/step - accuracy: 0.1277 - loss: 3.4658 - val_accuracy: 0.3069 - val_loss: 2.7148
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 143ms/step - accuracy: 0.2553 - loss: 2.9620 - val_accuracy: 0.4775 - val_loss: 2.0907
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 21s 150ms/step - accuracy: 0.3378 - loss: 2.4390 - val_accuracy: 0.6033 - val_loss: 1.4170
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 20s 145ms/step - accuracy: 0.4598 - loss: 1.9732 - val_accuracy: 0.6722 - val_loss: 1.2371
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 21s 149ms/step - accuracy: 0.5204 - loss: 1.7313 - val_accuracy: 0.7126 - val_loss: 1.0944
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - accuracy: 0.5572 - loss: 1.5723 - val_accuracy: 0.8054 - val_loss: 0.7680
Epoch 8/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 20s 148ms/step - accuracy: 0.6174 - loss: 1.3447 - val_accuracy: 0.830

In [147]:
model.evaluate(X_test,y_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.8516 - loss: 0.5961


[0.586044192314148, 0.8441246747970581]

In [149]:
test_path=r'/kaggle/input/traffic-sign/traffic_Data/TEST'
original_test=[]
for filename in os.listdir(test_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        img = cv2.imread(os.path.join(test_path, filename))
        original_test.append(img)
#a= sorted([len(original_test[i]) for i in range(len(original_test))] )
#original_test
#y_val = model.predict(original_test)
#df = pd.to_csv('path') -> df[col] = Y_val

In [150]:
original_test=np.array(list(map(preprocessing,original_test)))  
original_test=original_test.reshape(original_test.shape[0],original_test.shape[1],original_test.shape[2],1)

In [152]:
y_val = model.predict(original_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step


In [153]:
val = np.array([np.where(y_val[i] == sorted(y_val[i])[57])[0] for i in range(len(y_val))])
val

array([[49],
       [13],
       [ 6],
       ...,
       [ 0],
       [27],
       [16]])

In [154]:
y_val.shape

(1994, 58)

In [156]:
df = pd.DataFrame()
df['img'] = os.listdir(test_path)
df['label'] = val
df
df.to_csv('/kaggle/working/t.csv',index = False )

In [157]:
d = pd.read_csv('/kaggle/working/t.csv')
d

,img,label
0,043_0030_j.png,49
1,013_0033.png,13
2,002_1_0001_1_j.png,6
3,039_1_0013_1_j.png,43
4,016_0005_j.png,16
...,...,...
1989,046_0007_j.png,45
1990,003_1_0031.png,3
1991,000_1_0006_1_j.png,0
1992,027_1_0011_1_j.png,27
